# Campus Forest vs Non-Forest Classification 🌲

This notebook performs binary land cover classification (Forest vs Non-Forest) using Sentinel-2 satellite imagery and Google Earth Engine.

In [ ]:
import os
from dotenv import load_dotenv
import ee
import geemap

# Load environment variables
load_dotenv()

# Explicitly specify the project parameter in ee.Initialize()
ee_project = os.getenv('EE_PROJECT_ID')
if not ee_project:
    raise ValueError("EE_PROJECT_ID not set in .env file")

ee.Initialize(project=ee_project)

### 2. Load Campus Boundary

In [ ]:
campus_geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [80.01710357666015, 23.173962177472703],
            [80.03259601593017, 23.165361215115187],
            [80.03654422760009, 23.172502420044232],
            [80.026802444458, 23.181694681000845],
            [80.01542987823485, 23.176960548201308],
        ]
    ]
}
campus = ee.Geometry(campus_geojson)

### 3. Load Sentinel-2 & Cloud Mask

In [ ]:
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud = 1 << 10
    cirrus = 1 << 11
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(cirrus).eq(0))
    return image.updateMask(mask).divide(10000)

dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate('2025-11-15', '2026-01-15')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
           .map(mask_s2_clouds))

image = dataset.median().clip(campus)

### 4. Add NDVI

In [ ]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
image = image.addBands(ndvi)
bands = ['B4', 'B8', 'NDVI']

### 5. Load Training Points
**Note:** Ensure 'users/cosypix/forest_points' and 'users/cosypix/non_forest_points' exist in your Earth Engine Assets.

In [ ]:
try:
    forest_points = ee.FeatureCollection('users/cosypix/forest_points')
    non_forest_points = ee.FeatureCollection('users/cosypix/non_forest_points')
    training_points = forest_points.merge(non_forest_points)
except Exception as e:
    print("Error loading assets. Make sure you have uploaded the training data.")
    print(e)

### 6. Sample Training Data

In [ ]:
# Check if training_points exists before running sampleRegions
if 'training_points' in locals():
    training = image.select(bands).sampleRegions(
        collection=training_points,
        properties=['label'],
        scale=10
    )

    training = training.filter(ee.Filter.notNull(bands + ['label']))

### 7. Train/Test Split

In [ ]:
if 'training' in locals():
    training = training.randomColumn('random')
    train_set = training.filter(ee.Filter.lt('random', 0.7))
    test_set = training.filter(ee.Filter.gte('random', 0.7))

### 8. Train Random Forest

In [ ]:
if 'train_set' in locals():
    classifier = ee.Classifier.smileRandomForest(150).train(
        features=train_set,
        classProperty='label',
        inputProperties=bands
    )

### 9. Accuracy Assessment

In [ ]:
if 'test_set' in locals() and 'classifier' in locals():
    validated = test_set.classify(classifier)
    confusion_matrix = validated.errorMatrix('label', 'classification')
    print(confusion_matrix.getInfo())
    print("Accuracy:", confusion_matrix.accuracy().getInfo())
    print("Kappa:", confusion_matrix.kappa().getInfo())

### 10. Classify Campus

In [ ]:
if 'classifier' in locals():
    classified = image.select(bands).classify(classifier)
    smooth_classification = classified.focal_mode(1)

### 11. Visualize

In [ ]:
Map = geemap.Map()
Map.centerObject(campus, 16)

if 'smooth_classification' in locals():
    Map.addLayer(smooth_classification,
                 {'min':0,'max':1,'palette':['lightgray','darkgreen']},
                 'Forest Map')
else:
    # Fallback visualization if classification failed (e.g. no training data)
    Map.addLayer(image, {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3}, 'Campus RGB')

Map